# Homework 2


Hồ Thanh Nhân - 21127122

---

In [30]:
import numpy as np

[3, 4]

In [33]:
N = 1000 # 1000 virtual fair coins
T = 10 # 10 times of flipping each coin independently
states = ['H', 'T'] # 2 states of flipping coin. Head and Tail
coins = {}

def experiments():
    # Flipping 1000 virtual fair coins. Flip each coin independently 10 times.
    for i in range(N):
        coins[f'c{i + 1}'] = [np.random.choice(states) for _ in range(T)]

    # The first coin flipped
    c_1 = coins['c1']

    # A coin chosen randomly from 1000 coins
    c_rand = coins[f'c{np.random.randint(N) + 1}']

    # c_min is the coin which had the minimum frequency of heads
    min_head = c_1.count('H')
    c_min = c_1
    for i in range(N - 1):
        count_tmp = coins[f'c{i + 2}'].count('H')
        if count_tmp < min_head:
            min_head = count_tmp
            c_min = coins[f'c{i + 2}']

    v_1 = c_1.count('H') / 10
    v_rand = c_rand.count('H') / 10
    v_min = c_min.count('H') / 10

    print(v_1)
    print(v_rand)
    print(v_min)

0.5
0.5
0.0


## 1. 
Giải thích ...

Do đó, đáp án đúng là: [d] (i) Not learning, (ii) Supervised Learning, (iii) Reinforcement Learning.


## 2.

## 3.

## 4.

## 5.

## 6.

## 7-10. 

### Import các thư viện cần thiết

In [4]:
import numpy as np

### Định nghĩa các hàm

Hàm phát sinh ra `target_w`, véc-tơ tham số của $f$

In [5]:
def generate_target_w():
    """
    Generates target_w from two random, uniformly distributed points in [-1, 1] x [-1, 1].
    
    Returns
    -------
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    """
    # Generate two points from a uniform distribution over [-1, 1]x[-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    # Compute the target W from these two points
    target_w = np.array([p1[1] * p2[0] - p1[0] * p2[1], p2[1] - p1[1], p1[0] - p2[0]]).reshape((-1, 1))
    
    return target_w

Hàm phát sinh ra tập dữ liệu

In [6]:
def generate_data(N, target_w):
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : int
        The number of examples.
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    
    Returns
    -------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.        
    """
    X = np.random.uniform(-1, 1, (N, 2))
    X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
    Y = np.sign(np.dot(X, target_w))
    
    return X, Y

Hàm chạy PLA

In [7]:
def run_PLA(X, Y):
    """
    Runs PLA.
    
    Parameters
    ----------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.
    
    Returns
    -------
    w : numpy array, shape (3, 1) 
        The vector of parameters of g.
    num_iterations : int
        The number of iterations PLA takes to converge.
    """
    w = np.zeros((X.shape[1], 1)) # Init w
    iteration = 0
    
    # TODO
    
    return w, iteration

Hàm main

In [8]:
def main(N):
    """
    Parameters
    ----------
    N : int
        The number of training examples.
    """
    num_runs = 1000
    avg_num_iterations = 0.0 # The average number of iterations PLA takes to converge
    avg_test_err = 0.0 # The average test error of g - the final hypothesis picked by PLA
    
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, Y = generate_data(N, target_w)
        
        # Run PLA to pick g
        w, num_iterations = run_PLA(X, Y)
        
        # Generate test set
        X_test, Y_test = generate_data(10000, target_w)
        
        # Test g
        test_err = np.mean(np.sign(np.dot(X_test, w)) != Y_test)
        
        # Update average values
        avg_num_iterations += (num_iterations * 1.0 / num_runs)
        avg_test_err += (test_err * 1.0 / num_runs)
    
    # Print results
    print('avg_num_iterations = %f' % (avg_num_iterations))
    print('avg_test_err = %f' % (avg_test_err))

### Chạy

In [9]:
main(N=10) # We can use `main(10)`, but `main(N=10)` is clearer

avg_num_iterations = 0.000000
avg_test_err = 1.000000


Câu 7: ta thấy kết quả gần nhất với đáp án [b] 15.

Câu 8: ta thấy kết quả gần nhất với đáp án [c] 0.1.

In [10]:
main(N=100)

avg_num_iterations = 0.000000
avg_test_err = 1.000000


Câu 9: ta thấy kết quả gần nhất với đáp án ...

Câu 10: ta thấy kết quả gần nhất với đáp án ...